# <a href="https://colab.research.google.com/github/GR-Tang/IBM-AI0403-Team3/blob/main/TelcoChurnV10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the libraries needed for fiddling with the dataset
import pixiedust
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
sns.set_theme(style="darkgrid")

In [ ]:
#importing the necessary libraries for later
#!pip install imblearn
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import plot_tree
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from scipy.stats import sem
import graphviz
from sklearn.ensemble import GradientBoostingClassifier 
from lightgbm import LGBMClassifier
import lightgbm as lgbm

In [ ]:
#getting the dataset and loading it up
raw_df=pixiedust.sampleData('https://raw.githubusercontent.com/GR-Tang/IBM-AI0403-Team3/main/telco-data.csv')

In [ ]:
#checking and verifying the dataset
raw_df.info()

In [ ]:
#getting the headers to fiddle with the features
raw_df.head()

In [ ]:
#can skip, for visualisation
display(raw_df)

In [ ]:
#can skip, for visualisation
#raw_df.groupby('Churn')[['MonthlyContract', 'OneYearContract', 'TwoYearContract']].sum()
#raw_df.groupby('Churn')[['PhoneService', 'MultipleLines', 'DeviceProtection']].sum()
#raw_df.groupby('Churn')[['InternetService', 'FiberOptic', 'DSL']].sum()
#raw_df.groupby('Churn')[['OnlineSecurity', 'OnlineBackup', 'TechSupport']].sum()
#raw_df.groupby('Churn')[['StreamingTV', 'StreamingMovies']].sum()
#raw_df.groupby('Churn')[['PayByBankTransfer', 'PayByCC', 'PayByElectronicCheque', 'PayByMailedCheque']].sum()

In [ ]:
#one-hot codes in case needed
raw_df['SeniorCitizen']=raw_df['SeniorCitizen'].map({1:'Yes', 0:'No'})
raw_df['MultipleLines']=raw_df['MultipleLines'].map({'Yes':'Yes', 'No':'No','No phone service':'No'})
raw_df['OnlineSecurity']=raw_df['OnlineSecurity'].map({'Yes':'Yes', 'No':'No','No internet service':'No'})
raw_df['OnlineBackup']=raw_df['OnlineBackup'].map({'Yes':'Yes', 'No':'No','No internet service':'No'})
raw_df['DeviceProtection']=raw_df['DeviceProtection'].map({'Yes':'Yes', 'No':'No','No internet service':'No'})
raw_df['TechSupport']=raw_df['TechSupport'].map({'Yes':'Yes', 'No':'No','No internet service':'No'})
raw_df['StreamingTV']=raw_df['StreamingTV'].map({'Yes':'Yes', 'No':'No','No internet service':'No'})
raw_df['StreamingMovies']=raw_df['StreamingMovies'].map({'Yes':'Yes', 'No':'No','No internet service':'No'})
raw_df['InternetService']=raw_df['InternetService'].map({'DSL':'DSL', 'No':'No','Fiber optic':'Fiber_Optic'})
raw_df['Contract']=raw_df['Contract'].map({'Month-to-month':'Month_to_month', 'One year':'One_year','Two year':'Two_year'})
raw_df['PaymentMethod']=raw_df['PaymentMethod'].map({'Bank transfer (automatic)':'Bank_transfer_automatic',
                                                     'Credit card (automatic)':'Credit_card_automatic',
                                                     'Electronic check':'Electronic_cheque','Mailed check':'Mailed_cheque'})

raw_df=pd.get_dummies(raw_df, columns=['gender','Partner','SeniorCitizen','Dependents','PhoneService','MultipleLines',
                                       'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport',
                                       'StreamingTV','StreamingMovies','Contract','PaperlessBilling',
                                       'PaymentMethod'], dtype='int64')

#keeping Churn as binary class due to it being target feature
raw_df['Churn']=raw_df['Churn'].map({'Yes': 1,'No': 0})
raw_df['TotalCharges']=raw_df['TotalCharges'].replace({' ':'0'}).astype(float)
raw_df.drop(columns=['customerID'], inplace = True)

In [ ]:
#grouping tenure and monthlycharges into bins
raw_df['Bintenure']=(raw_df['tenure']/6).apply(np.ceil).astype("int64")
raw_df['BinMonthlyCharges']=(raw_df['MonthlyCharges']/10).apply(np.ceil).astype("int64")
raw_df['BinTotalCharges']=(raw_df['TotalCharges']/500).apply(np.ceil).astype("int64")

In [ ]:
#shifting the target column to position 0 for ease of reference later
churn_df = raw_df['Churn']
raw_df.drop(columns=['Churn'], inplace = True)
raw_df.insert(0, 'Churn', churn_df)

In [ ]:
#dropping unwanted columns
imbaldrop2bin_df= raw_df.drop(columns=['gender_Male', 'gender_Female','MonthlyCharges','TotalCharges','tenure'])
imbaldrop7bin_df=imbaldrop2bin_df.drop(columns=['PhoneService_Yes','PhoneService_No','StreamingTV_Yes','StreamingTV_No','StreamingMovies_Yes','StreamingMovies_No','MultipleLines_Yes','MultipleLines_No','BinTotalCharges'])

In [ ]:
imbaldrop2bin_df.head()

In [ ]:
#noticed target data is imbalanced, we can either undersample (randomly reduce negatives to match the number of positives) 
#or oversample (randomly generate synthetic positives to match the number of negatives)
#however, almost all articles points to balancing after splitting into test/train sets. 
pd.value_counts(raw_df['Churn'])

In [ ]:
#Selecting the CSV file to use
raw_df=imbaldrop2bin_df

In [ ]:
#taking 10% of data as test data, the rest to train 
train_df, test_df = train_test_split(raw_df, test_size=0.1, stratify=raw_df['Churn'], random_state=55)

#printing to check size
print("Size of the training dataset = ", train_df.shape)
print("Size of the testing dataset = ", test_df.shape)

#show sample of the dataset to verify
print("\n\nSample of the training dataset \n")
train_df.head()

In [ ]:
pd.value_counts(train_df['Churn'])

In [ ]:
#plot and show the current imbalance
rcParams['figure.figsize'] = 6,5
sns.countplot(x='Churn', data=train_df)
plt.title('Imbalanced Churns')
plt.show()

#undersampling NOTE - Choose the one below or this, do not use both
#shuffle the Dataset. Everyday we're shuffling..shuffling... shuffling... (frac=1 means gimme back all the rows)
shuffled_df = train_df.sample(frac=1,random_state=555)

#pull out all the churn positives
churnPos_df = shuffled_df.loc[shuffled_df['Churn'] == 1]

#randomly pick 1682 rows from the ChurnNegatives (majority)
churnNeg_df = shuffled_df.loc[shuffled_df['Churn'] == 0].sample(n = 1682, random_state = 55)

#joining them back again
train_df = pd.concat([churnNeg_df, churnPos_df])


In [ ]:
#oversampling NOTE - Choose the one above or this, do not use both
sm = SMOTE(sampling_strategy=0.6667, random_state=55)

oversam_train_X, oversam_train_Y = sm.fit_sample(train_df.drop('Churn', axis=1), train_df['Churn'])
train_df = pd.concat([pd.DataFrame(oversam_train_Y), pd.DataFrame(oversam_train_X)], axis=1)

In [ ]:
pd.value_counts(train_df['Churn'])

In [ ]:
#Now plot and show the corrected balance
rcParams['figure.figsize'] = 6,5
sns.countplot(x='Churn', data=train_df)
plt.title('Balanced Churns')
plt.show()

In [ ]:
#define target column for both test and train dataset
train_X, train_Y = train_df.iloc[:,1:],train_df.iloc[:,0]
test_X, test_Y = test_df.iloc[:,1:],test_df.iloc[:,0]

In [ ]:
#selecting the model, parameters used for the first run is to suppress error messages
model1 = xgb.XGBClassifier(use_label_encoder=False, verbosity=0)

#fitting the model to the training dataset
model1.fit(train_X, train_Y)

In [ ]:
def assess_model(model, test_X, test_Y, folds):
    #calculating 
    model_preds = model.predict(test_X)
    accuracy = accuracy_score(test_Y, model_preds)
    model_probs = model.predict_proba(test_X)
    model_probs = model_probs[:, 1]
    model_auc = roc_auc_score(test_Y, model_probs)
    model_fpr, model_tpr, _ = roc_curve(test_Y, model_probs)
    model_precision, model_recall, _ = precision_recall_curve(test_Y, model_probs)
    model_f1 = f1_score(test_Y, model_preds)
    score = cross_val_score(model, train_X, train_Y, cv=folds, scoring='roc_auc')
 
    print('Model: %s\n' % (model))
    print((folds),'Folds Cross Validation ROC AUC Scores: ', (score))
    print('Mean ROC AUC score: {0:.3f} (+/-{1:.3f})'.format(np.mean(score), sem(score)))
    print('\nTest Set score:')
    print('Accuracy: %.2f' % (accuracy * 100.0))
    print('ROC AUC=%.3f' % (model_auc))
    print('F1=%.3f' % (model_f1))

    
    #plotting confusion matrix
    rcParams['figure.figsize'] = 6,5
    fig, ax = plt.subplots()
    cm = confusion_matrix(test_Y, model_preds)
    tp = cm[1,1]
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    sns.heatmap(cm, annot=True, fmt="d", linewidths=.5, ax = ax)
    ax.set_title('Confusion Matrix')
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("Actual class")
    plt.show()
    #plot roc/precision/recall
    rcParams['figure.figsize'] = 10,8
    plt.plot(model_fpr, model_tpr, marker='.', label='ROC')
    plt.plot(model_recall, model_precision, marker='.', label='Recall/Precision')
    plt.xlabel('Recall\nFalse Positive Rate')
    plt.ylabel('Precision\nTrue Positive Rate')
    plt.legend()
    plt.show()

    #plot feature importance (since we are only using two models, using try/except)
    rcParams['figure.figsize'] = 8,8
    feat_importances = pd.Series(model.feature_importances_, index=train_X.columns)
    feat_importances.nlargest(38).plot(kind='barh').invert_yaxis()
 

#calculate mean square error, not sure if useful for binary classification, included for fun 
#MSE = np.sqrt(mean_squared_error(test_Y, xgb1_pred_Y))
#print("XGBoost1 MSE: %f" % (MSE))

In [ ]:
#visualising the decision tree for shits and giggles
rcParams['figure.figsize'] = 30,40
xgb.plot_tree(model1, rankdir='LR')
plt.show()

In [ ]:
assess_model(model1,test_X,test_Y,3)

In [ ]:
#this will take very very long (bob marley:"I've been watching you~~") to run due to the various combinations
#thought of increasing cores used but refrained, for portability

model2 = xgb.XGBClassifier(use_label_encoder=False, verbosity=0)

param_grid = {'eta':[0.1, 0.2, 0.3, 0.4, 0.5],
              'max_depth':[3, 4, 5, 6, 7],
              'gamma':[0, 3, 6, 9, 12],
              'subsample':[0.1, 0.25, 0.5, 0.75, 1]
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=55)
grid_search = GridSearchCV(model2, param_grid, scoring='roc_auc', cv=kfold)
grid_result = grid_search.fit(train_X, train_Y)

# summarize results
print("Best ROC AUC: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("ROC AUC %f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Refit the model with new optimised parameters
model3 = xgb.XGBClassifier(use_label_encoder=False, verbosity=0, **grid_result.best_params_)

#fitting the model to the training dataset
model3.fit(train_X, train_Y)

#assess the model with optimised parameters
assess_model(model3,test_X,test_Y,3)

In [ ]:
#selecting the alternative model and fitting it with train data, using default params 
model4 = LGBMClassifier()
model4.fit(train_X, train_Y)

In [ ]:
#assess the default LGBM model to get a baseline scoring
assess_model(model4,test_X,test_Y,3)

In [ ]:
#perform gridsearchcv LGBM model using a few parameters, 
#this process takes very long since it will run all the possible different combinations of the parameters
#params which default is better: 'boosting_type':['gbdt', 'rf'],'learning_rate':[0.025, 0.050, 0.075, 0.1, 0.2]

model5 = LGBMClassifier(verbosity=-1)

param_grid = {'learning_rate':[0.025, 0.05, 0.1, 0.2, 0.3],
              'extra_trees':[True, False],
              'max_bin':[10, 55, 155, 255, 280],
              'max_delta_step':[-1, 10, 20, 30, 50]
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=55)
grid_search = GridSearchCV(model5, param_grid, scoring='roc_auc', cv=kfold)
grid_result = grid_search.fit(train_X, train_Y)

# summarize results
print("Best ROC AUC: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("ROC AUC %f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#todo - fit lgbm model with optimised parameters 
model6 = LGBMClassifier(verbosity=-1, **grid_result.best_params_)
model6.fit(train_X, train_Y)

In [ ]:
#todo - assess updated LGBM model with optimised parameters
assess_model(model6,test_X,test_Y,3)

In [ ]:
model7 = GradientBoostingClassifier()
model7.fit(train_X, train_Y)

In [ ]:
#todo - assess updated LGBM model with optimised parameters
assess_model(model7,test_X,test_Y,3)

In [ ]:
#perform gridsearch for GBclassifier model using a few parameters, 
#this process takes very long since it will run all the possible different combinations of the parameters

model8 = GradientBoostingClassifier()

param_grid = {'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3],
              'n_estimators':[80, 90, 100, 110, 120],
              'subsample':[0.01, 0.25, 0.5, 0.75, 1.0],
              'max_depth':[1, 2, 3, 5, 7]
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=55)
grid_search = GridSearchCV(model8, param_grid, scoring='roc_auc', cv=kfold)
grid_result = grid_search.fit(train_X, train_Y)

# summarize results
print("Best ROC AUC: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("ROC AUC %f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model9 = GradientBoostingClassifier(**grid_result.best_params_)
model9.fit(train_X, train_Y)
assess_model(model9,test_X,test_Y,3)

model10=xgb.XGBClassifier(use_label_encoder=False, verbosity=0, eta=0.1, max_depth=4, gamma=3, subsample=0.75)

model10.fit(train_X, train_Y)

%%writefile telco_churn_class.py
from bentoml import BentoService, api, env, artifacts
from bentoml.adapters import DataframeInput
from bentoml.service.artifacts.common import PickleArtifact

@artifacts([PickleArtifact('model')])
@env(conda_dependencies=["scikit-learn"])


class TelcoChurnClassifier(BentoService):

    @api(input=DataframeInput(), batch=True)
    def predict(self, df):
        return self.artifacts.model.predict(df)


from telco_churn_class import TelcoChurnClassifier

# Create a iris classifier service instance
telco_churn_class_service = TelcoChurnClassifier()

# Pack the newly trained model artifact
telco_churn_class_service.pack('model', model10)

# Save the prediction service to disk for model serving
saved_path = telco_churn_class_service.save()

$ az login

$ az group create --name telco-classi --location southeastasia

#output
{
  "id": "/subscriptions/e4737090-78c3-4a83-b843-f2c83847cf52/resourceGroups/telco-classi",
  "location": "southeastasia",
  "managedBy": null,
  "name": "telco-classi",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


$ az acr create --resource-group telco-classi --name telcobentoclassifier --sku Basic --admin-enabled true

#output
{
  "adminUserEnabled": true,
  "creationDate": "2021-02-20T07:12:31.268475+00:00",
  "dataEndpointEnabled": false,
  "dataEndpointHostNames": [],
  "encryption": {
    "keyVaultProperties": null,
    "status": "disabled"
  },
  "id": "/subscriptions/e4737090-78c3-4a83-b843-f2c83847cf52/resourceGroups/telco-classi/providers/Microsoft.ContainerRegistry/registries/telcobentoclassifier",
  "identity": null,
  "location": "southeastasia",
  "loginServer": "telcobentoclassifier.azurecr.io",
  "name": "telcobentoclassifier",
  "networkRuleBypassOptions": "AzureServices",
  "networkRuleSet": null,
  "policies": {
    "quarantinePolicy": {
      "status": "disabled"
    },
    "retentionPolicy": {
      "days": 7,
      "lastUpdatedTime": "2021-02-20T07:12:33.396227+00:00",
      "status": "disabled"
    },
    "trustPolicy": {
      "status": "disabled",
      "type": "Notary"
    }
  },
  "privateEndpointConnections": [],
  "provisioningState": "Succeeded",
  "publicNetworkAccess": "Enabled",
  "resourceGroup": "telco-classi",
  "sku": {
    "name": "Basic",
    "tier": "Basic"
  },
  "status": null,
  "storageAccount": null,
  "systemData": {
    "createdAt": "2021-02-20T07:12:31.268475+00:00",
    "createdBy": "nightsteed@hotmail.com",
    "createdByType": "User",
    "lastModifiedAt": "2021-02-20T07:12:31.268475+00:00",
    "lastModifiedBy": "nightsteed@hotmail.com",
    "lastModifiedByType": "User"
  },
  "tags": {},
  "type": "Microsoft.ContainerRegistry/registries",
  "zoneRedundancy": "Disabled"
}

$ az acr login --name telcobentoclassifier

#output
Login Succeeded

$ az acr show --name telcobentoclassifier --query loginServer --output table

#output
Result
-------------------------------
telcobentoclassifier.azurecr.io

$ saved_path=$(bentoml get TelcoChurnClassifier:latest --print-location --quiet)

$ docker build -t telcobentoclassifier.azurecr.io/telco_churn_class $saved_path

#output
[+] Building 476.9s (12/14)
 => => sha256:fc7a7a5ce7cd6a2b16477983c67a830e7ff1d4ad9188fda4a955422d48e9efe0 75.34MB / 75.34MB                                                   139.7s
 => => extracting sha256:68ced04f60ab5c7a5f1d0b0b4e7572c5a4c8cce44866513d30d9df1a15277d6b                                                            1.3s
[+] Building 477.0s (12/14)
 => => sha256:fc7a7a5ce7cd6a2b16477983c67a830e7ff1d4ad9188fda4a955422d48e9efe0 75.34MB / 75.34MB                                                   139.7s
 => => extracting sha256:68ced04f60ab5c7a5f1d0b0b4e7572c5a4c8cce44866513d30d9df1a15277d6b                                                            1.3s
 => => sha256:15cb6d6426181fb283e4b9512ba1aa5d5a32e76a9c8c887c94367593e6d3cb81 46.26MB / 46.26MB                                                   193.9s
 => => sha256:e72ae797889829323ba5bcd2ca83495816ea9b879e7d4fc976f9adb6ba801011 136.62MB / 136.62MB                                                 297.7s
 => => sha256:dba7275d2f4a6e9a0dfe0be2fadeb6cd18a4b70bbc354356d54acc9de608b6ea 44.62MB / 44.62MB                                                   270.3s
[+] Building 477.6s (12/14)
 => => sha256:fc7a7a5ce7cd6a2b16477983c67a830e7ff1d4ad9188fda4a955422d48e9efe0 75.34MB / 75.34MB                                                   139.7s
 => => extracting sha256:68ced04f60ab5c7a5f1d0b0b4e7572c5a4c8cce44866513d30d9df1a15277d6b                                                            1.3s
 => => sha256:15cb6d6426181fb283e4b9512ba1aa5d5a32e76a9c8c887c94367593e6d3cb81 46.26MB / 46.26MB                                                   193.9s
 => => sha256:e72ae797889829323ba5bcd2ca83495816ea9b879e7d4fc976f9adb6ba801011 136.62MB / 136.62MB                                                 297.7s
 => => sha256:dba7275d2f4a6e9a0dfe0be2fadeb6cd18a4b70bbc354356d54acc9de608b6ea 44.62MB / 44.62MB                                                   270.3s
[+] Building 477.8s (14/14)
 => => sha256:9c388eb6d33c40662539172f0d9a357287bd1cd171692ca5c08db2886bc527c3 80.22MB / 80.22MB                                                   180.1s
 => => sha256:32d11a4c418ae2124dc597daee535cfd258447ae74b3305b19c6d071681b6da9 2.01kB / 2.01kB                                                       0.0s
[+] Building 482.1s (15/15) FINISHED
 => [internal] load build definition from Dockerfile                                                                                                 0.1s
 => => transferring dockerfile: 1.28kB                                                                                                               0.0s
 => [internal] load .dockerignore                                                                                                                    0.1s
 => => transferring context: 2B                                                                                                                      0.0s
 => [internal] load metadata for docker.io/bentoml/model-server:0.11.0-py38                                                                          4.5s
 => [auth] bentoml/model-server:pull token for registry-1.docker.io                                                                                  0.0s
 => [1/9] FROM docker.io/bentoml/model-server:0.11.0-py38@sha256:32d11a4c418ae2124dc597daee535cfd258447ae74b3305b19c6d071681b6da9                  304.5s
 => => resolve docker.io/bentoml/model-server:0.11.0-py38@sha256:32d11a4c418ae2124dc597daee535cfd258447ae74b3305b19c6d071681b6da9                    0.0s
 => => sha256:9c388eb6d33c40662539172f0d9a357287bd1cd171692ca5c08db2886bc527c3 80.22MB / 80.22MB                                                   180.1s
 => => sha256:32d11a4c418ae2124dc597daee535cfd258447ae74b3305b19c6d071681b6da9 2.01kB / 2.01kB                                                       0.0s
 => => sha256:68ced04f60ab5c7a5f1d0b0b4e7572c5a4c8cce44866513d30d9df1a15277d6b 27.09MB / 27.09MB                                                    79.3s
 => => sha256:3878306313754b53a34614fa148f62554859f77ec0d828dd9c0e7245f31980ea 5.13kB / 5.13kB                                                       0.0s
 => => sha256:96cf53b3a9dd496f4c91ab86eeadca2c8a31210c2e5c2a82badbb0dcf8c8f76b 50.36MB / 50.36MB                                                   117.6s
 => => sha256:fc7a7a5ce7cd6a2b16477983c67a830e7ff1d4ad9188fda4a955422d48e9efe0 75.34MB / 75.34MB                                                   139.7s
 => => extracting sha256:68ced04f60ab5c7a5f1d0b0b4e7572c5a4c8cce44866513d30d9df1a15277d6b                                                            1.3s
 => => sha256:15cb6d6426181fb283e4b9512ba1aa5d5a32e76a9c8c887c94367593e6d3cb81 46.26MB / 46.26MB                                                   193.9s
 => => sha256:e72ae797889829323ba5bcd2ca83495816ea9b879e7d4fc976f9adb6ba801011 136.62MB / 136.62MB                                                 297.7s
 => => sha256:dba7275d2f4a6e9a0dfe0be2fadeb6cd18a4b70bbc354356d54acc9de608b6ea 44.62MB / 44.62MB                                                   270.3s
 => => extracting sha256:9c388eb6d33c40662539172f0d9a357287bd1cd171692ca5c08db2886bc527c3                                                            3.2s
 => => extracting sha256:96cf53b3a9dd496f4c91ab86eeadca2c8a31210c2e5c2a82badbb0dcf8c8f76b                                                            2.6s
 => => extracting sha256:fc7a7a5ce7cd6a2b16477983c67a830e7ff1d4ad9188fda4a955422d48e9efe0                                                            2.6s
 => => extracting sha256:15cb6d6426181fb283e4b9512ba1aa5d5a32e76a9c8c887c94367593e6d3cb81                                                            2.2s
 => => sha256:66cdeb8a960567e487bbd21ba4b9bb0c50b652383e99b640c36a027d3b58bb94 631B / 631B                                                         196.3s
 => => extracting sha256:e72ae797889829323ba5bcd2ca83495816ea9b879e7d4fc976f9adb6ba801011                                                            2.9s
 => => extracting sha256:dba7275d2f4a6e9a0dfe0be2fadeb6cd18a4b70bbc354356d54acc9de608b6ea                                                            2.3s
 => => extracting sha256:66cdeb8a960567e487bbd21ba4b9bb0c50b652383e99b640c36a027d3b58bb94                                                            0.0s
 => [internal] load build context                                                                                                                    0.1s
 => => transferring context: 224.43kB                                                                                                                0.0s
 => [2/9] COPY environment.yml requirements.txt setup.sh* bentoml-init.sh python_version* /bento/                                                    0.4s
 => [3/9] WORKDIR /bento                                                                                                                             0.1s
 => [4/9] COPY docker-entrypoint.sh /usr/local/bin/                                                                                                  0.1s
 => [5/9] COPY environment.yml bundled_pip_dependencies*  /bento/bundled_pip_dependencies/                                                           0.1s
 => [6/9] RUN rm /bento/bundled_pip_dependencies/environment.yml                                                                                     0.3s
 => [7/9] RUN chmod +x /bento/bentoml-init.sh /usr/local/bin/docker-entrypoint.sh                                                                    0.5s
 => [8/9] RUN if [ -f /bento/bentoml-init.sh ]; then bash -c /bento/bentoml-init.sh; fi                                                            167.0s
 => [9/9] COPY . /bento                                                                                                                              0.1s
 => exporting to image                                                                                                                               4.3s
 => => exporting layers                                                                                                                              4.2s
 => => writing image sha256:d33df16e8c7b6b220b90441560a8bc005fd1004dd1312326fd8589ceea0c081b                                                         0.0s
 => => naming to telcobentoclassifier.azurecr.io/telco_churn_class

$ docker push telcobentoclassifier.azurecr.io/telco_churn_class

#output
Using default tag: latest
The push refers to repository [telcobentoclassifier.azurecr.io/telco_churn_class]
...
latest: digest: sha256:34a2c05c614c8fab4fdf6338c879d1c0ae49b0f365c9bc5c9b03f7660c213f2d size: 3674

$ az acr repository list --name telcobentoclassifier --output table

#output
Result
-----------------
telco_churn_class

$ az acr credential show -n telcobentoclassifier

#output
{
  "passwords": [
    {
      "name": "password",
      "value": "<removed>"
    },
    {
      "name": "password2",
      "value": "<removed>"
    }
  ],
  "username": "telcobentoclassifier"
}

$ az container create --resource-group telco-classi \
    --name telcobentoclassifier \
    --image telcobentoclassifier.azurecr.io/telco_churn_class \
    --cpu 1 \
    --memory 1 \
    --registry-login-server telcobentoclassifier.azurecr.io \
    --registry-username telcobentoclassifier \
    --registry-password <removed> \
    --dns-name-label telcobentoclassifier55 \
    --ports 5000

#output
{
  "containers": [
    {
      "command": null,
      "environmentVariables": [],
      "image": "telcobentoclassifier.azurecr.io/telco_churn_class",
      "instanceView": {
        "currentState": {
          "detailStatus": "",
          "exitCode": null,
          "finishTime": null,
          "startTime": "2021-02-20T07:47:43+00:00",
          "state": "Running"
        },
        "events": [
          {
            "count": 2,
            "firstTimestamp": "2021-02-20T07:45:46+00:00",
            "lastTimestamp": "2021-02-20T07:47:42+00:00",
            "message": "pulling image \"telcobentoclassifier.azurecr.io/telco_churn_class\"",
            "name": "Pulling",
            "type": "Normal"
          },
          {
            "count": 2,
            "firstTimestamp": "2021-02-20T07:47:33+00:00",
            "lastTimestamp": "2021-02-20T07:47:43+00:00",
            "message": "Successfully pulled image \"telcobentoclassifier.azurecr.io/telco_churn_class\"",
            "name": "Pulled",
            "type": "Normal"
          },
          {
            "count": 2,
            "firstTimestamp": "2021-02-20T07:47:35+00:00",
            "lastTimestamp": "2021-02-20T07:47:43+00:00",
            "message": "Created container",
            "name": "Created",
            "type": "Normal"
          },
          {
            "count": 2,
            "firstTimestamp": "2021-02-20T07:47:36+00:00",
            "lastTimestamp": "2021-02-20T07:47:43+00:00",
            "message": "Started container",
            "name": "Started",
            "type": "Normal"
          }
        ],
        "previousState": {
          "detailStatus": "Error",
          "exitCode": 3,
          "finishTime": "2021-02-20T07:47:41+00:00",
          "startTime": "2021-02-20T07:47:35+00:00",
          "state": "Terminated"
        },
        "restartCount": 1
      },
      "livenessProbe": null,
      "name": "telcobentoclassifier",
      "ports": [
        {
          "port": 5000,
          "protocol": "TCP"
        }
      ],
      "readinessProbe": null,
      "resources": {
        "limits": null,
        "requests": {
          "cpu": 1.0,
          "gpu": null,
          "memoryInGb": 1.0
        }
      },
      "volumeMounts": null
    }
  ],
  "diagnostics": null,
  "dnsConfig": null,
  "id": "/subscriptions/e4737090-78c3-4a83-b843-f2c83847cf52/resourceGroups/telco-classi/providers/Microsoft.ContainerInstance/containerGroups/telcobentoclassifier",
  "identity": null,
  "imageRegistryCredentials": [
    {
      "password": null,
      "server": "telcobentoclassifier.azurecr.io",
      "username": "telcobentoclassifier"
    }
  ],
  "instanceView": {
    "events": [],
    "state": "Running"
  },
  "ipAddress": {
    "dnsNameLabel": "telcobentoclassifier55",
    "fqdn": "telcobentoclassifier55.southeastasia.azurecontainer.io",
    "ip": "20.197.114.252",
    "ports": [
      {
        "port": 5000,
        "protocol": "TCP"
      }
    ],
    "type": "Public"
  },
  "location": "southeastasia",
  "name": "telcobentoclassifier",
  "networkProfile": null,
  "osType": "Linux",
  "provisioningState": "Succeeded",
  "resourceGroup": "telco-classi",
  "restartPolicy": "Always",
  "tags": {},
  "type": "Microsoft.ContainerInstance/containerGroups",
  "volumes": null
}

$ az container show --resource-group telco-classi --name telcobentoclassifier --query instanceView.state

#output
"Running"

$ curl -X \
    POST "telcobentoclassifier55.southeastasia.azurecontainer.io:5000/predict" \
    --header "Content-Type: application/json" \
    -d '[[1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,1,6,1]]'